## About

Create a CSV file for all the the analyses in the Empirical Datasets

## Imports

In [1]:
import pandas as pd
import plotly.express as px
from prettytable import PrettyTable
import plotly.graph_objects as go
import numpy as np
import random
import csv
import os
import json
from tqdm import tqdm
import math
from scipy.stats import wasserstein_distance

## Declares

In [2]:
# Local on Windows 10 box
WD = os.path.join("E:\\", "BUSTEDS-MH-develop")

tags = ["Empirical_14_datasets", "Empirical_Unmasked_Selectome_v6", "Empirical_Enard", 
        "Empirical_mtDNA_Invertebrate", "Empirical_mtDNA_Vertebrate", 
        "Empirical_Selectome_v7_Euteleostomi_unmasked", "Empirical_Shultz" ]

ER_Threshold = 5

FILES_WITH_ERRORS = []

## Helper functions

In [3]:
def read_json(filename):
    #print("# Reading:", filename)
    if os.stat(filename).st_size == 0: 
        #print("# -- Error -- file is empty:", filename)
        return []
    #end if
    
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    return json_data
#end method

# define function to calculate cv
#cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
cv = lambda x: np.std(x) / np.mean(x)

## Look over BUSTED[S]-MH Files

In [4]:
def get_BUSTEDS_MH_results(BUSTEDS_MH_DIR_FILES, tag, FILES_WITH_ERRORS):
    df_dict = {}
    for item in tqdm(BUSTEDS_MH_DIR_FILES):
        basename = os.path.basename(item).replace(".BUSTEDS-MH.json", "")
        
        # Read json file
        json_data_BUSTEDS_MH = read_json(item) # Returns "[]" if file is empty

        if json_data_BUSTEDS_MH == []: 
            FILES_WITH_ERRORS.append(item)
            continue
        #end if
        
        #print("# Data loaded")
        df_dict[basename] = {"Method": "BUSTEDS-MH"}
        df_dict[basename].update({"Sequences": json_data_BUSTEDS_MH["input"]["number of sequences"]})
        df_dict[basename].update({"Codons": json_data_BUSTEDS_MH["input"]["number of sites"]})
        df_dict[basename].update({"LRT p-value": json_data_BUSTEDS_MH["test results"]["p-value"]})

        # cAIC
        df_dict[basename].update({"cAIC": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["AIC-c"]})

        #Omegas and proportions
        data = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]
        w1 = round(data["0"]["omega"], 4)
        p1 = round(data["0"]["proportion"], 4)
        w2 = round(data["1"]["omega"], 4)
        p2 = round(data["1"]["proportion"], 4)
        w3 = round(data["2"]["omega"], 4)
        p3 = round(data["2"]["proportion"], 4)
        df_dict[basename].update({"w1": w1, "p1": p1})
        df_dict[basename].update({"w2": w2, "p2": p2})
        df_dict[basename].update({"w3": w3, "p3": p3})

        df_dict[basename].update({"CV(omega)": cv([w1, w2, w3])})

        # SRV rates and proportions
        data = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
        s1 = round(data["0"]["rate"], 4)
        s_p1 = round(data["0"]["proportion"], 4)
        s2 = round(data["1"]["rate"], 4)
        s_p2 = round(data["1"]["proportion"], 4)
        s3 = round(data["2"]["rate"], 4)
        s_p3 = round(data["2"]["proportion"], 4)
        df_dict[basename].update({"SRV1": s1, "SRV_p1": s_p1})
        df_dict[basename].update({"SRV2": s2, "SRV_p2": s_p2})
        df_dict[basename].update({"SRV3": s3, "SRV_p3": s3})
        df_dict[basename].update({"CV(alpha)": cv([s1, s2, s3])})

        # DH rate, TH rate, TH_SI rate
        df_dict[basename].update({"DH_Rate": float(json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                                  ["rate at which 2 nucleotides are changed instantly within a single codon"])})
        df_dict[basename].update({"TH_Rate": float(json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                                  ["rate at which 3 nucleotides are changed instantly within a single codon"])})
        df_dict[basename].update({"TH_Rate_SI": float(json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                                  ["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"])})

        # ER Sites, thresholded
        ER_SITES = []
        ER_df_dict = {}
        if "constrained" in json_data_BUSTEDS_MH["Evidence Ratios"].keys():
            #print("# ER Constrained Sites:", len(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]))
            for site, val in enumerate(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]):
                if val > ER_Threshold:
                    ER_SITES.append(str(site + 1))
                    ER_df_dict[site + 1] = {"BUSTEDS-MH ER": val}
                #end if
            #end for
            # add assert that there are more than 0 sites here.
            df_dict[basename].update({"BUSTEDS-MH_num_ER_Sites":  len(ER_df_dict.keys())})
            x = ER_df_dict.keys()
            x = [str(x) for x in x]
            df_dict[basename].update({"BUSTEDS-MH_ER_Sites":  "|".join(x)})
            #print(ER_df_dict.keys())
        #end if 
    # end for

    df_MH = pd.DataFrame.from_dict(df_dict, orient="index")
    df_MH = df_MH.reset_index()
    df_MH.index += 1
    df_MH.rename(columns={'index': 'Gene'}, inplace = True)
    df_MH["Dataset"] = tag
    return df_MH, FILES_WITH_ERRORS
#end method
    

## Look over BUSTED[S] Files

In [5]:

def get_BUSTEDS_results(BUSTEDS_DIR_FILES, tag, FILES_WITH_ERRORS):
    df_dict = {}
    for item in tqdm(BUSTEDS_DIR_FILES):
        basename = os.path.basename(item).replace(".nex.BUSTEDS.json", "")
        # Read json
        
        json_data_BUSTEDS = read_json(item)
        #print("# Data loaded:", item)

        if json_data_BUSTEDS == []: 
            FILES_WITH_ERRORS.append(item)
            continue
        #end if
        
        df_dict[basename] = {"Method": "BUSTEDS"}
        df_dict[basename].update({"Sequences": json_data_BUSTEDS["input"]["number of sequences"]})
        df_dict[basename].update({"Codons": json_data_BUSTEDS["input"]["number of sites"]})
        df_dict[basename].update({"LRT p-value": json_data_BUSTEDS["test results"]["p-value"]})

        # cAIC
        df_dict[basename].update({"cAIC": json_data_BUSTEDS["fits"]["Unconstrained model"]["AIC-c"]})

        A = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
        B = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
        C = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
        df_dict[basename].update({"CV(omega)": cv([A, B, C])})

        D = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
        E = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
        F = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
        df_dict[basename].update({"CV(alpha)": cv([D, E, F])})

        #Omegas and proportions
        data = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]
        w1 = round(data["0"]["omega"], 4)
        p1 = round(data["0"]["proportion"], 4)
        w2 = round(data["1"]["omega"], 4)
        p2 = round(data["1"]["proportion"], 4)
        w3 = round(data["2"]["omega"], 4)
        p3 = round(data["2"]["proportion"], 4)
        df_dict[basename].update({"w1": w1, "p1": p1})
        df_dict[basename].update({"w2": w2, "p2": p2})
        df_dict[basename].update({"w3": w3, "p3": p3})

        # SRV rates and proportions
        data = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
        s1 = round(data["0"]["rate"], 4)
        s_p1 = round(data["0"]["proportion"], 4)
        s2 = round(data["1"]["rate"], 4)
        s_p2 = round(data["1"]["proportion"], 4)
        s3 = round(data["2"]["rate"], 4)
        s_p3 = round(data["2"]["proportion"], 4)
        df_dict[basename].update({"SRV1": s1, "SRV_p1": s_p1})
        df_dict[basename].update({"SRV2": s2, "SRV_p2": s_p2})
        df_dict[basename].update({"SRV3": s3, "SRV_p3": s3})

        # ER Sites
        ER_SITES = []
        ER_df_dict = {}

        if "constrained" in json_data_BUSTEDS["Evidence Ratios"].keys():
            #print("# ER Constrained Sites:", len(json_data_BUSTEDS["Evidence Ratios"]["constrained"][0]))
            for site, val in enumerate(json_data_BUSTEDS["Evidence Ratios"]["constrained"][0]):
                if val > ER_Threshold:
                    ER_SITES.append(str(site + 1))
                    ER_df_dict[site + 1] = {"BUSTEDS ER": val}
                #end if
            #end for
            #df_dict[basename].update({"num_ER_Sites":  int(len(ER_df_dict.keys()))})
            df_dict[basename].update({"BUSTEDS_num_ER_Sites":  len(ER_df_dict.keys())})
            x = ER_df_dict.keys()
            x = [str(x) for x in x]
            df_dict[basename].update({"BUSTEDS_ER_Sites":  "|".join(x)})
            #print(ER_df_dict.keys())
        #end if   


    # End for

    df = pd.DataFrame.from_dict(df_dict, orient="index")
    df = df.reset_index()
    df.index += 1
    df.rename(columns={'index': 'Gene'}, inplace = True)
    df["Dataset"] = tag
    return df, FILES_WITH_ERRORS
#end method



## Calculate cAIC statistics

In [6]:
def calculate_statistics(df, df_MH):
    df["ΔcAIC"] = ""
    df["RelativeSupport"] = ""
    df["ER_Sites_Intersection"] = ""
    for index, row in tqdm(df_MH.iterrows()):
        gene = row["Gene"]
        MH_cAIC = float(row["cAIC"])

        BUSTEDS_row = df[df["Gene"] == row["Gene"]]
        index_BUSTEDS = df[df["Gene"] == row["Gene"]].index

        try:
            BUSTEDS_cAIC = float(BUSTEDS_row["cAIC"])
        except:
            #print(BUSTEDS_row)
            pass
        #end try

        BUSTEDSMH_w1, BUSTEDSMH_w2, BUSTEDSMH_w3 = float(row["w1"]), float(row["w2"]), float(row["w3"])
        try:
            BUSTEDS_w1, BUSTEDS_w2, BUSTEDS_w3 = float(BUSTEDS_row["w1"]), float(BUSTEDS_row["w2"]), float(BUSTEDS_row["w3"])
        except:
            #print(BUSTEDS_row["w1"], BUSTEDS_row["w2"], BUSTEDS_row["w3"])
            continue
        #end try

        BUSTEDSMH_p1, BUSTEDSMH_p2, BUSTEDSMH_p3 = float(row["p1"]), float(row["p2"]), float(row["p3"])
        BUSTEDS_p1, BUSTEDS_p2, BUSTEDS_p3 = float(BUSTEDS_row["p1"]), float(BUSTEDS_row["p2"]), float(BUSTEDS_row["p3"])

        BUSTEDSMH_s1, BUSTEDSMH_s2, BUSTEDSMH_s3 = float(row["SRV1"]), float(row["SRV2"]), float(row["SRV3"])
        BUSTEDS_s1, BUSTEDS_s2, BUSTEDS_s3 = float(BUSTEDS_row["SRV1"]), float(BUSTEDS_row["SRV2"]), float(BUSTEDS_row["SRV3"])

        BUSTEDSMH_sp1, BUSTEDSMH_sp2, BUSTEDSMH_sp3 = float(row["SRV_p1"]), float(row["SRV_p2"]), float(row["SRV_p3"])
        BUSTEDS_sp1, BUSTEDS_sp2, BUSTEDS_sp3 = float(BUSTEDS_row["SRV_p1"]), float(BUSTEDS_row["SRV_p2"]), float(BUSTEDS_row["SRV_p3"])

        #print(float(BUSTEDS_cAIC["cAIC"]))
        best_model = min(MH_cAIC, BUSTEDS_cAIC)
        #print()
        #print("# Gene:", row["Gene"])

        if BUSTEDS_cAIC == best_model:
            which_is_best = "BUSTEDS"
            delta_cAIC = MH_cAIC - best_model
            relative_support = math.exp(-delta_cAIC/2)
            # add to table
            #df.at['C', 'x'] = 10
            # 
            df.at[index_BUSTEDS, "ΔcAIC"] = delta_cAIC
            df.at[index_BUSTEDS, "RelativeSupport"] = relative_support
        elif MH_cAIC == best_model:
            which_is_best = "BUSTEDS-MH"
            delta_cAIC = BUSTEDS_cAIC - best_model
            relative_support = math.exp(-delta_cAIC/2)
            df_MH.at[index, "ΔcAIC"] = delta_cAIC
            df_MH.at[index, "RelativeSupport"] = relative_support
        else:
            pass
        #end if
        #print("# Best model is:", best_model, which_is_best, "by", delta_cAIC)
        #print("# With relative support:", relative_support)

        # Intersections of ER Sites.
        #print("# Examining ER Sites")
        # BUSTEDS-MH_ER_Sites
        # BUSTEDS_ER_Sites
        try:
            BUSTEDS_MH_ER_Sites = row["BUSTEDS-MH_ER_Sites"].split("|")
            BUSTEDS_df = df[df["Gene"] == row["Gene"]]
            BUSTEDS_ER_Sites    = BUSTEDS_df["BUSTEDS_ER_Sites"].tolist()[0].split("|")
            #print(BUSTEDS_MH_ER_Sites, BUSTEDS_ER_Sites)
            intersection = set(BUSTEDS_MH_ER_Sites).intersection(BUSTEDS_ER_Sites)
            #print(intersection)
            df.at[index, "ER_Sites_Intersection"] = "|".join(intersection)
        except:
            #print("ERROR --", row["BUSTEDS-MH_ER_Sites"])
            pass
        #end try


        # Wasserstein Distance (Earth Movers)
        #print("Omegas:", BUSTEDSMH_w1, BUSTEDSMH_w2, BUSTEDSMH_w3, BUSTEDS_w1, BUSTEDS_w2, BUSTEDS_w3)
        WD_unweighted_omega = wasserstein_distance([BUSTEDSMH_w1, BUSTEDSMH_w2, BUSTEDSMH_w3],
                                                   [BUSTEDS_w1, BUSTEDS_w2, BUSTEDS_w3])

        WD_weighted_omega   = wasserstein_distance([BUSTEDSMH_w1, BUSTEDSMH_w2, BUSTEDSMH_w3], 
                                                   [BUSTEDS_w1, BUSTEDS_w2, BUSTEDS_w3],
                                                   [BUSTEDSMH_p1, BUSTEDSMH_p2, BUSTEDSMH_p3],
                                                   [BUSTEDS_p1, BUSTEDS_p2, BUSTEDS_p3])

        #print("SRV:", BUSTEDSMH_s1, BUSTEDSMH_s2, BUSTEDSMH_s3, BUSTEDS_s1, BUSTEDS_s2, BUSTEDS_s3)

        WD_unweighted_srv   = wasserstein_distance([BUSTEDSMH_s1, BUSTEDSMH_s2, BUSTEDSMH_s3], 
                                                   [BUSTEDS_s1, BUSTEDS_s2, BUSTEDS_s3])

        WD_weighted_srv     = wasserstein_distance([BUSTEDSMH_s1, BUSTEDSMH_s2, BUSTEDSMH_s3], 
                                                   [BUSTEDS_s1, BUSTEDS_s2, BUSTEDS_s3], 
                                                   [BUSTEDSMH_sp1, BUSTEDSMH_sp2, BUSTEDSMH_sp3], 
                                                   [BUSTEDS_sp1, BUSTEDS_sp2, BUSTEDS_sp3])

        df_MH.at[index, "WD_unweighted(omega)"] = WD_unweighted_omega
        df_MH.at[index, "WD_weighted(omega)"] = WD_weighted_omega
        df_MH.at[index, "WD_unweighted(srv)"] = WD_unweighted_srv
        df_MH.at[index, "WD_weighted(srv)"] = WD_weighted_srv
    #end for
    return df, df_MH
#end method
    

## Concat tables


In [7]:
def concat_tables(df, df_MH):
    result = ""
    result = pd.concat([df_MH, df])
    result = result.fillna("")
    result = result.sort_values(by=["Gene", "Method"], ascending=True)
    result = result.reset_index(drop=True)
    result.index += 1
    return result
#end method

## Main

In [ ]:
for tag in tags:
    print("#", "="*80)
    print("# Processing files in:", tag)
    
    # Additional declares
    BUSTEDS_DIR = os.path.join(WD, "analysis", tag, "BUSTEDS")
    BUSTEDS_MH_DIR = os.path.join(WD, "analysis", tag, "BUSTEDS-MH")
    OUTPUT_CSV = os.path.join(WD, "tables", "Table_" + tag.upper() + ".csv")
    
    # Get list of files
    BUSTEDS_DIR_FILES = [os.path.join(BUSTEDS_DIR, file.name) for file in os.scandir(BUSTEDS_DIR) if file.name.endswith(".json")]
    BUSTEDS_MH_DIR_FILES = [os.path.join(BUSTEDS_MH_DIR, file.name) for file in os.scandir(BUSTEDS_MH_DIR) if file.name.endswith(".json")]

    # Report to user
    print("# Number of BUSTEDS results:", len(BUSTEDS_DIR_FILES))
    print("# Number of BUSTEDS-MH results:", len(BUSTEDS_MH_DIR_FILES))
    
    # Process files separately
    print("# Processing BUSTED[S] results")
    df, FILES_WITH_ERRORS    = get_BUSTEDS_results(BUSTEDS_DIR_FILES, tag, FILES_WITH_ERRORS)
    print("# Processing BUSTED[S]-MH results")
    df_MH, FILES_WITH_ERRORS = get_BUSTEDS_MH_results(BUSTEDS_MH_DIR_FILES, tag, FILES_WITH_ERRORS)
    
    # Calculate pairwise statistics
    print("# Calculating pairwise statistics")
    df, df_MH = calculate_statistics(df, df_MH)
    
    # Combine results
    print("# Formatting output")
    results = concat_tables(df, df_MH)
    
    # Save to file
    print("# Saving results to:", OUTPUT_CSV)
    results.to_csv(OUTPUT_CSV, index=False)
    
    # Files with errors
    print("# Error log...")
    print(len(FILES_WITH_ERRORS), "files had errors")

    with open("ERRORS_" + tag + ".log", "w") as fh:
        for file in FILES_WITH_ERRORS:
            print(file, file=fh)
        #end for
    #end with
    print("#", "="*80)
    print()
#end for

print("# Done processing")

# ================================================================================
# Processing files in: Empirical_14_datasets
# Number of BUSTEDS results: 14
# Number of BUSTEDS-MH results: 14
# Processing BUSTED[S] results


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:01<00:00,  9.32it/s]


# Processing BUSTED[S]-MH results


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 368.50it/s]


# Calculating pairwise statistics


14it [00:00, 933.88it/s]


# Formatting output
# Saving results to: E:\BUSTEDS-MH-develop\tables\Table_EMPIRICAL_14_DATASETS.csv
# Error log...
0 files had errors
# ================================================================================

# ================================================================================
# Processing files in: Empirical_Unmasked_Selectome_v6
# Number of BUSTEDS results: 13312
# Number of BUSTEDS-MH results: 13312
# Processing BUSTED[S] results


 47%|███████████████████████████████████▌                                        | 6238/13312 [00:11<00:15, 461.47it/s]

## End of file

In [ ]:
# Note Negative delta LL are convergence problems

In [ ]:
# Lower AIC values indicate a better-fit model, and a model with a delta-AIC (the difference between the two AIC values being compared) of more than -2 is considered significantly better than the model it is being compared to

In [ ]:
# Earth Mover's (Kantorovich) distance between two distrbuitions if you want a single number